In [ ]:
pip install ortools

In [2]:
import pandas as pd
import sys 
import os 
import numpy as np
from datetime import datetime

In [18]:
client_to_client = pd.read_csv('../data/distance_matrix_client_to_client.csv')
inter_to_client = pd.read_csv('../data/distance_matrix_inter_to_client.csv')

In [19]:
client_to_client

,ID Client 1,ID Client 2,geometry_client_1,geometry_client_2,distance_car,duration_car,distance_bike,duration_bike
0,559475456,559277088,POINT (1.3756893 48.7210523),POINT (1.3756893 48.7210523),0,0,0,0
1,559475456,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13809,1070,12103,2655
2,559277088,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13809,1070,12103,2655
3,559475456,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5880,469,4215,1092
4,559277088,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5880,469,4215,1092
...,...,...,...,...,...,...,...,...
6898,570590313,621952854,POINT (1.3579211 48.7447023),POINT (1.351704 48.7428046),805,149,581,142
6899,167937489,621952854,POINT (1.3475838 48.7428036),POINT (1.351704 48.7428046),501,96,501,84
6900,474721965,621952854,POINT (1.3706525 48.7406195),POINT (1.351704 48.7428046),2320,405,2320,637
6901,751167409,621952854,POINT (1.3388591 48.7293302),POINT (1.351704 48.7428046),2400,356,3488,697


In [5]:
inter_to_client

,ID Intervenant,ID Client,geometry_intervenant,Permis,geometry_client,distance,duration
0,162858075,559475456,POINT (1.3746477 48.7222691),1.0,POINT (1.3756893 48.7210523),375,83
1,78007018,559475456,POINT (1.351054 48.689927),1.0,POINT (1.3756893 48.7210523),5454,497
2,856089133,559475456,POINT (1.5230773 48.6454179),0.0,POINT (1.3756893 48.7210523),17631,3520
3,842621873,559475456,POINT (1.3798422 48.728863),1.0,POINT (1.3756893 48.7210523),1732,250
4,838320706,559475456,POINT (1.3919715 48.7385156),1.0,POINT (1.3756893 48.7210523),3884,326
...,...,...,...,...,...,...,...
2827,818696864,621952854,POINT (1.3579211 48.7447023),1.0,POINT (1.351704 48.7428046),805,149
2828,813991780,621952854,POINT (1.0731951 48.6927148),1.0,POINT (1.351704 48.7428046),23526,1463
2829,739888851,621952854,POINT (1.303906 48.75186),1.0,POINT (1.351704 48.7428046),5135,362
2830,78012267,621952854,POINT (1.3719853 48.7292064),1.0,POINT (1.351704 48.7428046),3234,530


In [6]:
parent_directory = os.path.dirname(os.getcwd())
sys.path.append(parent_directory)  

from utils.utils import load_data
from utils.utils import parse_duration, determine_time_window, prestation_duration

In [7]:
data = load_data()

df = data['JAN24']

In [8]:
for i in range(len(df)):
    heure_debut = df.at[i, 'Heure de début']
    if df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 10:
        df.at[i, 'Prestation'] = 'PDJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 14 and heure_debut.hour>12 :
        df.at[i, 'Prestation'] = 'DEJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS':
        df.at[i, 'Prestation'] = 'DIN'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour < 12:
        df.at[i, 'Prestation'] = 'TOILETTE_MAT'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour > 12:
        df.at[i, 'Prestation'] = 'TOILETTE_SOIR'
    

In [9]:
data_day = df[df['Date']=='2024-01-01'].copy()

In [10]:
prestation_duration_list = []
for i in range(len(data_day)):
    h1 = data_day.iloc[i].loc['Heure de début']
    h2 = data_day.iloc[i].loc['Heure de fin']

    prestation_duration_list.append(prestation_duration(h1,h2))

data_day['Prestation_duration'] = prestation_duration_list

In [11]:
data_day['time_window'] = data_day['Prestation'].apply(determine_time_window)

In [12]:
data_day

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation,Prestation_duration,time_window
0,559475456,162858075,2024-01-01,07:15:00,07:45:00,PDJ,30.0,"(7, 9)"
1,559277088,162858075,2024-01-01,07:45:00,08:30:00,TOILETTE_MAT,45.0,"(7, 10)"
2,87852633,78007018,2024-01-01,07:45:00,08:30:00,TOILETTE_MAT,45.0,"(7, 10)"
3,243033408,810259688,2024-01-01,07:45:00,08:15:00,TOILETTE_MAT,30.0,"(7, 10)"
4,814940942,710283561,2024-01-01,07:45:00,09:20:00,TOILETTE_MAT,95.0,"(7, 10)"
...,...,...,...,...,...,...,...,...
61,714782168,78007018,2024-01-01,19:00:00,20:00:00,DIN,60.0,"(19, 21)"
62,814940942,810259688,2024-01-01,19:00:00,20:00:00,TOILETTE_SOIR,60.0,"(18, 20)"
63,530138806,480302361,2024-01-01,19:15:00,19:45:00,DIN,30.0,"(19, 21)"
64,803656603,856089133,2024-01-01,19:55:00,20:25:00,TOILETTE_SOIR,30.0,"(18, 20)"


In [20]:
matrix_size = len(data_day) + len(data_day['ID Intervenant'].unique())
intervention_intervention_matrix = np.zeros((matrix_size, matrix_size))

for i in range(0,matrix_size):
    for j in range(i,matrix_size): 
        
        if ((i<len(data_day['ID Intervenant'].unique())) & (j<(len(data_day['ID Intervenant'].unique()))) & (i!=j)):
            intervention_intervention_matrix[i,j] = 10000
        
        if ((i<len(data_day['ID Intervenant'].unique())) & (j>=len(data_day['ID Intervenant'].unique()))):
            intervenant_i = data_day['ID Intervenant'].unique()[i]
            intervention_j = data_day['ID Client'].iloc[j-len(data_day['ID Intervenant'].unique())]

            row_1 = inter_to_client[(inter_to_client['ID Client'] == intervention_j) & (inter_to_client['ID Intervenant'] == intervenant_i)]

            if not row_1.empty:
                distance = row_1['duration'].iloc[0]
                intervention_intervention_matrix[i, j] = distance/60
            
            else:
                    print(intervenant_i, intervention_j)
                    intervention_intervention_matrix[i, j] = np.nan

    
        if ((i>=len(data_day['ID Intervenant'].unique())) & (j>=len(data_day['ID Intervenant'].unique()))):
            intervention_i = data_day['ID Client'].iloc[i-len(data_day['ID Intervenant'].unique())]
            intervention_j = data_day['ID Client'].iloc[j-len(data_day['ID Intervenant'].unique())]

            if intervention_i == intervention_j:
                intervention_intervention_matrix[i,j] = 0
            
            else:
                row_1 = client_to_client[(client_to_client['ID Client 1'] == intervention_i) & (client_to_client['ID Client 2'] == intervention_j)]
                row_2 = client_to_client[(client_to_client['ID Client 1'] == intervention_j) & (client_to_client['ID Client 2'] == intervention_i)]
                
                if not row_1.empty:
                    distance = row_1['duration_car'].iloc[0]
                    intervention_intervention_matrix[i, j] = distance//60

                elif not row_2.empty:
                    distance = row_2['duration_car'].iloc[0]
                    intervention_intervention_matrix[i, j] = distance/60

                else:
                    print(intervention_i, intervention_j)
                    intervention_intervention_matrix[i, j] = np.nan  
                
for i in range(matrix_size):
    for j in range(i):
        intervention_intervention_matrix[i, j] = intervention_intervention_matrix[j, i]

In [45]:
intervention_intervention_matrix

array([[0.00000000e+00, 1.00000000e+04, 1.00000000e+04, ...,
        9.73333333e+00, 2.95000000e+00, 5.53333333e+00],
       [1.00000000e+04, 0.00000000e+00, 1.00000000e+04, ...,
        1.02000000e+01, 9.53333333e+00, 9.21666667e+00],
       [1.00000000e+04, 1.00000000e+04, 0.00000000e+00, ...,
        2.17833333e+01, 1.12166667e+01, 1.05166667e+01],
       ...,
       [9.73333333e+00, 1.02000000e+01, 2.17833333e+01, ...,
        0.00000000e+00, 9.55000000e+00, 4.00000000e+00],
       [2.95000000e+00, 9.53333333e+00, 1.12166667e+01, ...,
        9.55000000e+00, 0.00000000e+00, 5.00000000e+00],
       [5.53333333e+00, 9.21666667e+00, 1.05166667e+01, ...,
        4.00000000e+00, 5.00000000e+00, 0.00000000e+00]])

In [26]:
time_window_intervenant = [(6,22)]*len(data_day['ID Intervenant'].unique().reshape(-1,1)) 

[(6, 22),
 (6, 22),
 (6, 22),
 (6, 22),
 (6, 22),
 (6, 22),
 (6, 22),
 (6, 22),
 (7, 9),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 9),
 (7, 9),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 9),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (7, 10),
 (19, 21),
 (7, 10),
 (7, 10),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (19, 21),
 (18, 20),
 (18, 20),
 (7, 22),
 (19, 21),
 (18, 20),
 (19, 21),
 (19, 21),
 (18, 20),
 (19, 21),
 (19, 21),
 (18, 20),
 (18, 20),
 (18, 20),
 (19, 21),
 (18, 20),
 (19, 21),
 (19, 21),
 (18, 20),
 (19, 21),
 (18, 20),
 (19, 21),
 (18, 20),
 (18, 20)]

In [49]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = intervention_intervention_matrix
    time_window_intervenant = [(6,22)]*len(data_day['ID Intervenant'].unique().reshape(-1,1))
    data["time_windows"] = time_window_intervenant + data_day['time_window'].to_list()
    data["time_windows"] = list(data["time_windows"])
    data["num_vehicles"] = len(data_day['ID Intervenant'].unique())
    data['starts'] = list(np.arange(0, len(data_day['ID Intervenant'].unique())))
    data['ends'] = list(np.arange(0, len(data_day['ID Intervenant'].unique())))
    data["starts"] = [int(start) for start in data["starts"]]
    data["ends"] = [int(end) for end in data["ends"]]

    return(data)

# Create and register a transit callback.
def time_callback(data, from_index, to_index, manager):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["time_matrix"][from_node][to_node]

"""def print_solution(data, manager, routing, solution):
    """""""Prints solution on console."""""""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")"""


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index_1 = index
            index = solution.Value(routing.NextVar(index))
            print(time_callback(data, index_1,index, manager))

        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )

        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        1000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data["time_windows"]):
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(7, 22)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 0
0.0
Route for vehicle 0:
0 Time(7,7) -> 0 Time(7,7)
Time of the route: 7min

0.0
Route for vehicle 1:
1 Time(7,7) -> 1 Time(7,7)
Time of the route: 7min

0.0
Route for vehicle 2:
2 Time(7,7) -> 2 Time(7,7)
Time of the route: 7min

0.0
Route for vehicle 3:
3 Time(7,7) -> 3 Time(7,7)
Time of the route: 7min

0.0
Route for vehicle 4:
4 Time(7,7) -> 4 Time(7,7)
Time of the route: 7min

0.0
Route for vehicle 5:
5 Time(7,7) -> 5 Time(7,7)
Time of the route: 7min

20.416666666666668
10.0
5.0
24.0
27.0
6.0
5.0
17.0
12.633333333333333
5.0
18.0
24.0
10.0
4.65
3.0
18.083333333333332
22.0
8.0
8.0
22.0
19.516666666666666
16.0
37.61666666666667
19.0
9.0
27.366666666666667
25.0
7.0
21.0
17.0
0.0
16.833333333333332
Route for vehicle 6:
6 Time(7,7) -> 39 Time(7,7) -> 38 Time(7,7) -> 36 Time(7,7) -> 35 Time(7,7) -> 34 Time(7,7) -> 33 Time(7,7) -> 32 Time(7,7) -> 31 Time(7,7) -> 30 Time(7,7) -> 29 Time(7,7) -> 28 Time(7,7) -> 27 Time(7,7) -> 26 Time(7,7) -> 25 Time(7,7) -> 24 Time(7,7) -> 23